In [168]:
import pandas as pd
import numpy as np



-----------PLAN:

cross-validation:
splitting the original df into 5 folds each containing the same amount of data
than running the ranking codes on each of the groups, getting different rankings
-> run ranking with eacch of the 5 folds -> 5 rankings
-> do statistical evaluation using spearman´s rank correlation, kendall´s tau-b or mean average precision

PROBLEM: test group and training group are not different -> doesn´t make sense to "test" like this


In [169]:
original_dms_data = pd.read_csv('/Users/liza/Desktop/Bioinfo Project/DMS_data/GFP_AEQVI_Sarkisyan_2016.csv')

num_folds = 5  # Number of folds to create

# Shuffle the DataFrame randomly
shuffled_original_df = original_dms_data.sample(frac=1, random_state=42)

# Calculate the number of rows per fold
fold_size = len(original_dms_data) // num_folds

# Split the DataFrame into folds
folds = {}
for i in range(num_folds-1):
    fold = shuffled_original_df.iloc[i*fold_size:(i+1)*fold_size]
    folds[f"Fold_{i+1}"] = fold
remaining_fold = shuffled_original_df.iloc[(num_folds-1)*fold_size:]
folds[f"Fold_{num_folds}"] = remaining_fold

fold_1_dms = folds["Fold_1"]
fold_2_dms = folds["Fold_2"]
fold_3_dms = folds["Fold_3"]
fold_4_dms = folds["Fold_4"]
fold_5_dms = folds["Fold_5"]

let´s begin with proofing ranking 5
-> we have to rerun the whole code for ranking 5 on each fold to get 5 different rankings

In [171]:

# Assuming your DataFrame is named 'fold_1_dms_col'
fold_1_dms = fold_1_dms.copy()  # Make a copy of the DataFrame

# Split the 'mutant' column into multiple columns
split_columns_fold_1 = fold_1_dms['mutant'].str.split(':', expand=True)

# Generate column names from 'm1' to 'mN'
num_columns_fold_1 = split_columns_fold_1.shape[1]
column_names_fold_1 = [f"m{i+1}" for i in range(num_columns_fold_1)]

# Assign the split columns with the generated column names to the DataFrame
fold_1_dms = pd.concat([fold_1_dms, split_columns_fold_1], axis=1)
fold_1_dms.columns = list(fold_1_dms.columns[:len(fold_1_dms.columns)-num_columns_fold_1]) + column_names_fold_1

fold_1_dms = fold_1_dms.reset_index()
fold_1_dms = fold_1_dms.drop('index', axis=1)  # Remove the old index column if needed

print(fold_1_dms.index)

RangeIndex(start=0, stop=10342, step=1)


In [172]:
# how many mutations does each sequence contain?
list_mut_count_in_progress_fold_1 = []
for i in range(len(fold_1_dms['mutant'])):
    list_mut_count_in_progress_fold_1.append(fold_1_dms['mutant'].iloc[i].count(':'))
list_mut_count_prae_fold_1 = np.array(list_mut_count_in_progress_fold_1)
list_mut_count_fold_1 = (list_mut_count_prae_fold_1 + 1)
df_mutation_counts_fold_1 = pd.DataFrame(list_mut_count_fold_1)
print(df_mutation_counts_fold_1.index)

RangeIndex(start=0, stop=10342, step=1)


In [173]:
working_dataframe_fold_1_try = pd.concat([fold_1_dms, df_mutation_counts_fold_1], axis=1)
len(working_dataframe_fold_1_try)

10342

In [174]:
#more convenient df
working_dataframe_fold_1 = pd.concat([fold_1_dms, df_mutation_counts_fold_1[0]], axis=1)
columns_to_drop = ['mutant', 'mutated_sequence', 'DMS_score_bin']
working_dataframe_fold_1.drop(columns_to_drop, axis=1)
working_dataframe_fold_1.rename(columns={0: 'mut_count'}, inplace = True)
count_fscore_frame_fold_1 = working_dataframe_fold_1[['DMS_score', 'mut_count']]

In [175]:
count_fscore_frame_fold_1.head()

,DMS_score,mut_count
0,3.525239,3
1,1.548703,8
2,1.652061,10
3,3.565658,2
4,3.732474,5


In [176]:
#generating list that contains all possible mutations
import re

mutation_columns_fold_1 = [col for col in working_dataframe_fold_1.columns if re.match(r'^m\d+$', col)]
working_dataframe_only_ms_fold_1 = working_dataframe_fold_1.loc[:, mutation_columns_fold_1]
stacked_columns_fold_1 = working_dataframe_only_ms_fold_1.stack().tolist()
all_possible_mutations_fold_1 = list(set(stacked_columns_fold_1))
if 'None' in all_possible_mutations_fold_1:
    all_possible_mutations_fold_1.remove('None')


In [177]:
#dataframe which mutants contain the mutation
list_of_dfs_fold_1 = []


for i in all_possible_mutations_fold_1:
    new_column_name_fold_1 = f'{i}'
    new_column_values_fold_1 = [fold_1_dms["mutant"].str.contains(i, regex= False)]
    new_df_fold_1 = pd.DataFrame({new_column_name_fold_1: new_column_values_fold_1})
    new_df_exploded_fold_1 = new_df_fold_1.explode(new_column_name_fold_1)
    list_of_dfs_fold_1.append(new_df_exploded_fold_1)


result_how_often_fold_1 = pd.concat(list_of_dfs_fold_1, axis=1)
result_how_often_fold_1 = result_how_often_fold_1.reset_index(drop=True)


In [178]:
#variance and number of used values calculation

for j in range(2, 16):
    variance_per_mutant_list_fold_1 = []

    for i in all_possible_mutations_fold_1:
        mut_count_fscore_fold_1 = count_fscore_frame_fold_1.loc[result_how_often_fold_1[i] == True]
        fscore_mut_fold_1 = mut_count_fscore_fold_1['DMS_score'].loc[mut_count_fscore_fold_1['mut_count'] == j]
        varianz_mut_fold_1 = fscore_mut_fold_1.var()
        variance_per_mutant_list_fold_1.append(varianz_mut_fold_1)

    variance_per_mutant_series_fold_1 = pd.Series(variance_per_mutant_list_fold_1, index=all_possible_mutations_fold_1)
    variance_per_mutant_df_fold_1 = variance_per_mutant_series_fold_1.to_frame()

#how many data per mutation
    how_many_for_variance_fold_1 = []

    for i in all_possible_mutations_fold_1:
        mut_count_fscore_fold_1 = count_fscore_frame_fold_1.loc[result_how_often_fold_1[i] == True]
        fscore_mut_fold_1 = mut_count_fscore_fold_1['DMS_score'].loc[mut_count_fscore_fold_1['mut_count'] == j]
        wie_viel_jeweils_fold_1 = len(fscore_mut_fold_1)
        how_many_for_variance_fold_1.append(wie_viel_jeweils_fold_1)

    how_many_for_variance_fold_1 = pd.Series(how_many_for_variance_fold_1, index=all_possible_mutations_fold_1)
    how_many_for_variance_df_fold_1 = how_many_for_variance_fold_1.to_frame()


    how_many_AND_variance_df_fold_1 = pd.concat([how_many_for_variance_df_fold_1, variance_per_mutant_df_fold_1], axis = 1)
    how_many_AND_variance_df_fold_1.columns = ['Anzahl benutzter Werte', 'Varianz']
    how_many_AND_variance_df_fold_1 = how_many_AND_variance_df_fold_1.dropna()

In [179]:
#calculating the mean of the variances across all mutation counts for every possible mutation
frame_zum_mitteln_variance_fold_1 = pd.DataFrame(index = all_possible_mutations_fold_1)
variance_per_mutant_count_list_fold_1 = []

# ATTENTION!! only mut_counts from 2 to 8 used, because everything above is a "dead" protein
for j in range(2, 8):
    variance_per_mutant_list_fold_1 = []

    for i in all_possible_mutations_fold_1:
        mut_count_fscore_fold_1 = count_fscore_frame_fold_1.loc[result_how_often_fold_1[i] == True]
        fscore_mut_fold_1 = mut_count_fscore_fold_1['DMS_score'].loc[mut_count_fscore_fold_1['mut_count'] == j]
        varianz_mut_fold_1 = fscore_mut_fold_1.var()  #die varianz je mutation je anzahl
        variance_per_mutant_list_fold_1.append(varianz_mut_fold_1) #liste der Varianzen ALLER Mutationen je anzahl

    variance_per_mutant_df = pd.DataFrame(variance_per_mutant_list_fold_1, index=all_possible_mutations_fold_1)
    variance_per_mutant_count_list_fold_1.append(variance_per_mutant_df)
variance_per_mutant_count_df_fold_1 = pd.concat(variance_per_mutant_count_list_fold_1, axis=1)
variance_per_mutant_count_df_fold_1.set_axis(range(2,8), axis=1, inplace=True)


# dataframe with all variances per all mutations (rows) per all counts (columns)
mean_variances_per_mutations_fold_1 = pd.DataFrame(variance_per_mutant_count_df_fold_1.mean(axis=1, skipna=True), columns=['Mean'])


/var/folders/sk/q8h0c3zs1v9dt93y5m134swr0000gn/T/ipykernel_11103/902919963.py:18: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  variance_per_mutant_count_df_fold_1.set_axis(range(2,8), axis=1, inplace=True)


In [180]:
how_many_per_mutant_count_list_fold_1 = []

for j in range(2, 8):
    how_many_for_variance_fold_1 = []

    for i in all_possible_mutations_fold_1:
        mut_count_fscore_fold_1 = count_fscore_frame_fold_1.loc[result_how_often_fold_1[i] == True]
        fscore_mut_fold_1 = mut_count_fscore_fold_1['DMS_score'].loc[mut_count_fscore_fold_1['mut_count'] == j]
        wie_viel_jeweils_fold_1 = len(fscore_mut_fold_1)
        how_many_for_variance_fold_1.append(wie_viel_jeweils_fold_1)

    how_many_per_mutant_df_fold_1 = pd.DataFrame(how_many_for_variance_fold_1, index=all_possible_mutations_fold_1)
    how_many_per_mutant_count_list_fold_1.append(how_many_per_mutant_df_fold_1)
how_many_per_mutant_count_df_fold_1 = pd.concat(how_many_per_mutant_count_list_fold_1, axis=1)
how_many_per_mutant_count_df_fold_1.set_axis(range(2,8), axis=1, inplace=True)

mean_how_many_per_mutations_fold_1 = pd.DataFrame(how_many_per_mutant_count_df_fold_1.mean(axis=1, skipna=True), columns=['Mean'])

/var/folders/sk/q8h0c3zs1v9dt93y5m134swr0000gn/T/ipykernel_11103/2720770737.py:15: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  how_many_per_mutant_count_df_fold_1.set_axis(range(2,8), axis=1, inplace=True)


ALTERNATIVE CODE USING VECTORING: should be faster

nur_fscore_mut_count_fold_1 = working_dataframe_fold_1.loc[:, ["DMS_score", "mut_count"]]

mut_present_indices = result_how_often_fold_1.any(axis=1)
mut_absent_indices = ~mut_present_indices

only_rows_with_mut_fold_1 = nur_fscore_mut_count_fold_1[(nur_fscore_mut_count_fold_1['mut_count'] > 2) & mut_present_indices]
mean_dms_score_only_mut_fold_1 = only_rows_with_mut_fold_1['DMS_score'].mean()

only_rows_without_mut_fold_1 = working_dataframe_fold_1[(working_dataframe_fold_1['mut_count'] > 2) & mut_absent_indices]
mean_dms_score_without_mut_fold_1 = only_rows_without_mut_fold_1['DMS_score'].mean()

difference_means_fold_1 = mean_dms_score_only_mut_fold_1 - mean_dms_score_without_mut_fold_1

all_differences_means_fold_1 = pd.DataFrame({'Difference': difference_means_fold_1}, index=all_possible_mutations_fold_1)


In [181]:
#differences between all mutant-fscores WITH and WITHOUT the one specific mutation, takes loong
nur_fscore_mut_count_fold_1 = working_dataframe_fold_1.loc[:, ["DMS_score", "mut_count"]]
differences_list_fold_1 = []

for i in all_possible_mutations_fold_1:
    index_when_mut_present_fold_1 = result_how_often_fold_1.loc[result_how_often_fold_1[i] == True].index

    only_rows_with_mut_fold_1 = nur_fscore_mut_count_fold_1 [(nur_fscore_mut_count_fold_1 ['mut_count'] >2) & (nur_fscore_mut_count_fold_1 .index.isin(index_when_mut_present_fold_1))]

# Calculate the mean of DMS_score for the filtered rows
    mean_dms_score_only_mut_fold_1 = only_rows_with_mut_fold_1['DMS_score'].mean()
#-------------
    index_when_not_mut_present_fold_1 = result_how_often_fold_1.loc[result_how_often_fold_1[i] == False].index

    only_rows_withOUT_mut_fold_1 = working_dataframe_fold_1[(working_dataframe_fold_1['mut_count'] >2) & (working_dataframe_fold_1.index.isin(index_when_not_mut_present_fold_1))]

# Calculate the mean of DMS_score for the filtered rows
    mean_dms_score_every_but_mut_fold_1 = only_rows_withOUT_mut_fold_1['DMS_score'].mean()
#----------------
    difference_means_fold_1 = mean_dms_score_only_mut_fold_1 - mean_dms_score_every_but_mut_fold_1
    differences_list_fold_1.append(difference_means_fold_1)

all_differences_means_fold_1 = pd.DataFrame({'Difference': differences_list_fold_1}, index=all_possible_mutations_fold_1)


In [182]:
list_wie_oft_mut_fold_1 = []
for j in all_possible_mutations_fold_1:
    matching_indexes_fold_1 = result_how_often_fold_1.loc[result_how_often_fold_1[j] == True].index
    wie_oft_fold_1 = len(matching_indexes_fold_1)
    list_wie_oft_mut_fold_1.append(wie_oft_fold_1)
df_wie_oft_muts_insg_fold_1 = pd.DataFrame(list_wie_oft_mut_fold_1, index=all_possible_mutations_fold_1)
df_wie_oft_muts_insg_fold_1

,0
L60R,1
D103N,16
E124K,30
L236R,13
D129Y,2
...,...
F165I,19
G91R,2
D76A,6
P58L,10


In [183]:
list_ranking5_fold_1 = []
for i in all_possible_mutations_fold_1:

    score_ranking5_fold_1 = (all_differences_means_fold_1.loc[i].values[0] * df_wie_oft_muts_insg_fold_1.loc[i].values[0]* 51714) / np.sum(variance_per_mutant_count_df_fold_1.loc[i].values)
    list_ranking5_fold_1.append(score_ranking5_fold_1)
ranking5_unsorted_fold_1 = pd.DataFrame(list_ranking5_fold_1, index=all_possible_mutations_fold_1, columns=['ranking5_score_fold_1'])
ranking5_fold_1 = ranking5_unsorted_fold_1.sort_values(by='ranking5_score_fold_1', ascending= False)
print(ranking5_fold_1)

       ranking5_score_fold_1
V163A          964535.775073
D197G          846853.395101
I167V          838167.171190
K79R           789873.907796
K156R          674315.167722
...                      ...
F165I                    NaN
G91R                     NaN
D76A                     NaN
P58L                     NaN
K41L                     NaN

[1471 rows x 1 columns]



------------------------fold_2--------------------------


In [184]:

# Assuming your DataFrame is named 'fold_2_dms_col'
fold_2_dms = fold_2_dms.copy()  # Make a copy of the DataFrame

# Split the 'mutant' column into multiple columns
split_columns_fold_2 = fold_2_dms['mutant'].str.split(':', expand=True)

# Generate column names from 'm1' to 'mN'
num_columns_fold_2 = split_columns_fold_2.shape[1]
column_names_fold_2 = [f"m{i+1}" for i in range(num_columns_fold_2)]

# Assign the split columns with the generated column names to the DataFrame
fold_2_dms = pd.concat([fold_2_dms, split_columns_fold_2], axis=1)
fold_2_dms.columns = list(fold_2_dms.columns[:len(fold_2_dms.columns)-num_columns_fold_2]) + column_names_fold_2

fold_2_dms = fold_2_dms.reset_index()
fold_2_dms = fold_2_dms.drop('index', axis=1)  # Remove the old index column if needed

print(fold_2_dms.index)

RangeIndex(start=0, stop=10342, step=1)


In [185]:
# how many mutations does each sequence contain?
list_mut_count_in_progress_fold_2 = []
for i in range(len(fold_2_dms['mutant'])):
    list_mut_count_in_progress_fold_2.append(fold_2_dms['mutant'].iloc[i].count(':'))
list_mut_count_prae_fold_2 = np.array(list_mut_count_in_progress_fold_2)
list_mut_count_fold_2 = (list_mut_count_prae_fold_2 + 1)
df_mutation_counts_fold_2 = pd.DataFrame(list_mut_count_fold_2)
print(df_mutation_counts_fold_2.index)

RangeIndex(start=0, stop=10342, step=1)


In [186]:
working_dataframe_fold_2_try = pd.concat([fold_2_dms, df_mutation_counts_fold_2], axis=1)

In [187]:
#more convenient df
working_dataframe_fold_2 = pd.concat([fold_2_dms, df_mutation_counts_fold_2[0]], axis=1)
columns_to_drop = ['mutant', 'mutated_sequence', 'DMS_score_bin']
working_dataframe_fold_2.drop(columns_to_drop, axis=1)
working_dataframe_fold_2.rename(columns={0: 'mut_count'}, inplace = True)
count_fscore_frame_fold_2 = working_dataframe_fold_2[['DMS_score', 'mut_count']]

In [188]:
count_fscore_frame_fold_2.head()

,DMS_score,mut_count
0,1.306102,4
1,3.565222,2
2,3.431563,3
3,3.744799,2
4,1.736148,4


In [189]:
#generating list that contains all possible mutations
import re

mutation_columns_fold_2 = [col for col in working_dataframe_fold_2.columns if re.match(r'^m\d+$', col)]
working_dataframe_only_ms_fold_2 = working_dataframe_fold_2.loc[:, mutation_columns_fold_2]
stacked_columns_fold_2 = working_dataframe_only_ms_fold_2.stack().tolist()
all_possible_mutations_fold_2 = list(set(stacked_columns_fold_2))
if 'None' in all_possible_mutations_fold_2:
    all_possible_mutations_fold_2.remove('None')


In [190]:
#dataframe which mutants contain the mutation
list_of_dfs_fold_2 = []

# For-Loop zum Erstellen und Hinzufügen eines neuen Columns in jedem Durchlauf
for i in all_possible_mutations_fold_2:
    new_column_name_fold_2 = f'{i}'
    new_column_values_fold_2 = [fold_2_dms["mutant"].str.contains(i, regex= False)]
    new_df_fold_2 = pd.DataFrame({new_column_name_fold_2: new_column_values_fold_2})
    new_df_exploded_fold_2 = new_df_fold_2.explode(new_column_name_fold_2)
    list_of_dfs_fold_2.append(new_df_exploded_fold_2)

# Verketten der DataFrames in der Liste
result_how_often_fold_2 = pd.concat(list_of_dfs_fold_2, axis=1)
result_how_often_fold_2 = result_how_often_fold_2.reset_index(drop=True)


In [191]:
#Varianz berechnen lassen, mit wie vielen Werten wurde gerechnet

for j in range(2, 16):
    variance_per_mutant_list_fold_2 = []

    for i in all_possible_mutations_fold_2:
        mut_count_fscore_fold_2 = count_fscore_frame_fold_2.loc[result_how_often_fold_2[i] == True]
        fscore_mut_fold_2 = mut_count_fscore_fold_2['DMS_score'].loc[mut_count_fscore_fold_2['mut_count'] == j]
        varianz_mut_fold_2 = fscore_mut_fold_2.var()
        variance_per_mutant_list_fold_2.append(varianz_mut_fold_2)

    variance_per_mutant_series_fold_2 = pd.Series(variance_per_mutant_list_fold_2, index=all_possible_mutations_fold_2)
    variance_per_mutant_df_fold_2 = variance_per_mutant_series_fold_2.to_frame()

#rausfinden wie viele Daten wir jeweils haben zum berechnen
    how_many_for_variance_fold_2 = []

    for i in all_possible_mutations_fold_2:
        mut_count_fscore_fold_2 = count_fscore_frame_fold_2.loc[result_how_often_fold_2[i] == True]
        fscore_mut_fold_2 = mut_count_fscore_fold_2['DMS_score'].loc[mut_count_fscore_fold_2['mut_count'] == j]
        wie_viel_jeweils_fold_2 = len(fscore_mut_fold_2)
        how_many_for_variance_fold_2.append(wie_viel_jeweils_fold_2)

    how_many_for_variance_fold_2 = pd.Series(how_many_for_variance_fold_2, index=all_possible_mutations_fold_2)
    how_many_for_variance_df_fold_2 = how_many_for_variance_fold_2.to_frame()


    how_many_AND_variance_df_fold_2 = pd.concat([how_many_for_variance_df_fold_2, variance_per_mutant_df_fold_2], axis = 1)
    how_many_AND_variance_df_fold_2.columns = ['Anzahl benutzter Werte', 'Varianz']
    how_many_AND_variance_df_fold_2 = how_many_AND_variance_df_fold_2.dropna()

In [192]:
#calculating the mean of the variances across all mutation counts for every possible mutation
frame_zum_mitteln_variance_fold_2 = pd.DataFrame(index = all_possible_mutations_fold_2)
variance_per_mutant_count_list_fold_2 = []

# ACHTUNG: es werden nur counts von 2 bis 7 einbezogen weil die mit mehr sowieso "kaputt" sind!!!
for j in range(2, 8):
    variance_per_mutant_list_fold_2 = []

    for i in all_possible_mutations_fold_2:
        mut_count_fscore_fold_2 = count_fscore_frame_fold_2.loc[result_how_often_fold_2[i] == True]
        fscore_mut_fold_2 = mut_count_fscore_fold_2['DMS_score'].loc[mut_count_fscore_fold_2['mut_count'] == j]
        varianz_mut_fold_2 = fscore_mut_fold_2.var()  #die varianz je mutation je anzahl
        variance_per_mutant_list_fold_2.append(varianz_mut_fold_2) #liste der Varianzen ALLER Mutationen je anzahl

    variance_per_mutant_df = pd.DataFrame(variance_per_mutant_list_fold_2, index=all_possible_mutations_fold_2)
    variance_per_mutant_count_list_fold_2.append(variance_per_mutant_df)
variance_per_mutant_count_df_fold_2 = pd.concat(variance_per_mutant_count_list_fold_2, axis=1)
variance_per_mutant_count_df_fold_2.set_axis(range(2,8), axis=1, inplace=True)


# dataframe mit allen varianzen (Zellen) pro alle mutationen (rows) pro alle counts (columns)
mean_variances_per_mutations_fold_2 = pd.DataFrame(variance_per_mutant_count_df_fold_2.mean(axis=1, skipna=True), columns=['Mean'])


/var/folders/sk/q8h0c3zs1v9dt93y5m134swr0000gn/T/ipykernel_11103/2096673579.py:18: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  variance_per_mutant_count_df_fold_2.set_axis(range(2,8), axis=1, inplace=True)


In [193]:
how_many_per_mutant_count_list_fold_2 = []

for j in range(2, 8):
    how_many_for_variance_fold_2 = []

    for i in all_possible_mutations_fold_2:
        mut_count_fscore_fold_2 = count_fscore_frame_fold_2.loc[result_how_often_fold_2[i] == True]
        fscore_mut_fold_2 = mut_count_fscore_fold_2['DMS_score'].loc[mut_count_fscore_fold_2['mut_count'] == j]
        wie_viel_jeweils_fold_2 = len(fscore_mut_fold_2)
        how_many_for_variance_fold_2.append(wie_viel_jeweils_fold_2)

    how_many_per_mutant_df_fold_2 = pd.DataFrame(how_many_for_variance_fold_2, index=all_possible_mutations_fold_2)
    how_many_per_mutant_count_list_fold_2.append(how_many_per_mutant_df_fold_2)
how_many_per_mutant_count_df_fold_2 = pd.concat(how_many_per_mutant_count_list_fold_2, axis=1)
how_many_per_mutant_count_df_fold_2.set_axis(range(2,8), axis=1, inplace=True)

mean_how_many_per_mutations_fold_2 = pd.DataFrame(how_many_per_mutant_count_df_fold_2.mean(axis=1, skipna=True), columns=['Mean'])

/var/folders/sk/q8h0c3zs1v9dt93y5m134swr0000gn/T/ipykernel_11103/1600817455.py:15: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  how_many_per_mutant_count_df_fold_2.set_axis(range(2,8), axis=1, inplace=True)


In [194]:
#differences between all mutant-fscores WITH and WITHOUT the one specific mutation, takes loong
nur_fscore_mut_count_fold_2 = working_dataframe_fold_2.loc[:, ["DMS_score", "mut_count"]]
differences_list_fold_2 = []

for i in all_possible_mutations_fold_2:
    index_when_mut_present_fold_2 = result_how_often_fold_2.loc[result_how_often_fold_2[i] == True].index

    only_rows_with_mut_fold_2 = nur_fscore_mut_count_fold_2 [(nur_fscore_mut_count_fold_2 ['mut_count'] >2) & (nur_fscore_mut_count_fold_2 .index.isin(index_when_mut_present_fold_2))]

# Calculate the mean of DMS_score for the filtered rows
    mean_dms_score_only_mut_fold_2 = only_rows_with_mut_fold_2['DMS_score'].mean()
#-------------
    index_when_not_mut_present_fold_2 = result_how_often_fold_2.loc[result_how_often_fold_2[i] == False].index

    only_rows_withOUT_mut_fold_2 = working_dataframe_fold_2[(working_dataframe_fold_2['mut_count'] >2) & (working_dataframe_fold_2.index.isin(index_when_not_mut_present_fold_2))]

# Calculate the mean of DMS_score for the filtered rows
    mean_dms_score_every_but_mut_fold_2 = only_rows_withOUT_mut_fold_2['DMS_score'].mean()
#----------------
    difference_means_fold_2 = mean_dms_score_only_mut_fold_2 - mean_dms_score_every_but_mut_fold_2
    differences_list_fold_2.append(difference_means_fold_2)

all_differences_means_fold_2 = pd.DataFrame({'Difference': differences_list_fold_2}, index=all_possible_mutations_fold_2)


In [195]:
list_wie_oft_mut_fold_2 = []
for j in all_possible_mutations_fold_2:
    matching_indexes_fold_2 = result_how_often_fold_2.loc[result_how_often_fold_2[j] == True].index
    wie_oft_fold_2 = len(matching_indexes_fold_2)
    list_wie_oft_mut_fold_2.append(wie_oft_fold_2)
df_wie_oft_muts_insg_fold_2 = pd.DataFrame(list_wie_oft_mut_fold_2, index=all_possible_mutations_fold_2)
df_wie_oft_muts_insg_fold_2

,0
L60R,4
D103N,15
E124K,17
L236R,13
D129Y,6
...,...
F165I,29
M153E,2
D76A,7
P58L,12


In [196]:
list_ranking5_fold_2 = []
for i in all_possible_mutations_fold_2:

    score_ranking5_fold_2 = (all_differences_means_fold_2.loc[i].values[0] * df_wie_oft_muts_insg_fold_2.loc[i].values[0]* 51714) / np.sum(variance_per_mutant_count_df_fold_2.loc[i].values)
    list_ranking5_fold_2.append(score_ranking5_fold_2)
ranking5_unsorted_fold_2 = pd.DataFrame(list_ranking5_fold_2, index=all_possible_mutations_fold_2, columns=['ranking5_score_fold_2'])
ranking5_fold_2 = ranking5_unsorted_fold_2.sort_values(by='ranking5_score_fold_2', ascending= False)
print(ranking5_fold_2)

       ranking5_score_fold_2
V163A           1.042382e+06
D197G           7.713377e+05
E172G           7.485073e+05
I171V           7.448647e+05
S72G            7.105375e+05
...                      ...
F165I                    NaN
M153E                    NaN
D76A                     NaN
P58L                     NaN
Q80V                     NaN

[1447 rows x 1 columns]




-------------------fold 3 --------------------

In [197]:
# Assuming your DataFrame is named 'fold_3_dms_col'
fold_3_dms = fold_3_dms.copy()  # Make a copy of the DataFrame

# Split the 'mutant' column into multiple columns
split_columns_fold_3 = fold_3_dms['mutant'].str.split(':', expand=True)

# Generate column names from 'm1' to 'mN'
num_columns_fold_3 = split_columns_fold_3.shape[1]
column_names_fold_3 = [f"m{i+1}" for i in range(num_columns_fold_3)]

# Assign the split columns with the generated column names to the DataFrame
fold_3_dms = pd.concat([fold_3_dms, split_columns_fold_3], axis=1)
fold_3_dms.columns = list(fold_3_dms.columns[:len(fold_3_dms.columns)-num_columns_fold_3]) + column_names_fold_3

fold_3_dms = fold_3_dms.reset_index()
fold_3_dms = fold_3_dms.drop('index', axis=1)  # Remove the old index column if needed

print(fold_3_dms.index)
# how many mutations does each sequence contain?
list_mut_count_in_progress_fold_3 = []
for i in range(len(fold_3_dms['mutant'])):
    list_mut_count_in_progress_fold_3.append(fold_3_dms['mutant'].iloc[i].count(':'))
list_mut_count_prae_fold_3 = np.array(list_mut_count_in_progress_fold_3)
list_mut_count_fold_3 = (list_mut_count_prae_fold_3 + 1)
df_mutation_counts_fold_3 = pd.DataFrame(list_mut_count_fold_3)
print(df_mutation_counts_fold_3.index)
working_dataframe_fold_3_try = pd.concat([fold_3_dms, df_mutation_counts_fold_3], axis=1)
#more convenient df
working_dataframe_fold_3 = pd.concat([fold_3_dms, df_mutation_counts_fold_3[0]], axis=1)
columns_to_drop = ['mutant', 'mutated_sequence', 'DMS_score_bin']
working_dataframe_fold_3.drop(columns_to_drop, axis=1)
working_dataframe_fold_3.rename(columns={0: 'mut_count'}, inplace=True)
count_fscore_frame_fold_3 = working_dataframe_fold_3[['DMS_score', 'mut_count']]
count_fscore_frame_fold_3.head()
#generating list that contains all possible mutations
import re

mutation_columns_fold_3 = [col for col in working_dataframe_fold_3.columns if re.match(r'^m\d+$', col)]
working_dataframe_only_ms_fold_3 = working_dataframe_fold_3.loc[:, mutation_columns_fold_3]
stacked_columns_fold_3 = working_dataframe_only_ms_fold_3.stack().tolist()
all_possible_mutations_fold_3 = list(set(stacked_columns_fold_3))
if 'None' in all_possible_mutations_fold_3:
    all_possible_mutations_fold_3.remove('None')

#dataframe which mutants contain the mutation
list_of_dfs_fold_3 = []

# For-Loop zum Erstellen und Hinzufügen eines neuen Columns in jedem Durchlauf
for i in all_possible_mutations_fold_3:
    new_column_name_fold_3 = f'{i}'
    new_column_values_fold_3 = [fold_3_dms["mutant"].str.contains(i, regex=False)]
    new_df_fold_3 = pd.DataFrame({new_column_name_fold_3: new_column_values_fold_3})
    new_df_exploded_fold_3 = new_df_fold_3.explode(new_column_name_fold_3)
    list_of_dfs_fold_3.append(new_df_exploded_fold_3)

# Verketten der DataFrames in der Liste
result_how_often_fold_3 = pd.concat(list_of_dfs_fold_3, axis=1)
result_how_often_fold_3 = result_how_often_fold_3.reset_index(drop=True)

#Varianz berechnen lassen, mit wie vielen Werten wurde gerechnet

for j in range(2, 16):
    variance_per_mutant_list_fold_3 = []

    for i in all_possible_mutations_fold_3:
        mut_count_fscore_fold_3 = count_fscore_frame_fold_3.loc[result_how_often_fold_3[i] == True]
        fscore_mut_fold_3 = mut_count_fscore_fold_3['DMS_score'].loc[mut_count_fscore_fold_3['mut_count'] == j]
        varianz_mut_fold_3 = fscore_mut_fold_3.var()
        variance_per_mutant_list_fold_3.append(varianz_mut_fold_3)

    variance_per_mutant_series_fold_3 = pd.Series(variance_per_mutant_list_fold_3, index=all_possible_mutations_fold_3)
    variance_per_mutant_df_fold_3 = variance_per_mutant_series_fold_3.to_frame()

    #rausfinden wie viele Daten wir jeweils haben zum berechnen
    how_many_for_variance_fold_3 = []

    for i in all_possible_mutations_fold_3:
        mut_count_fscore_fold_3 = count_fscore_frame_fold_3.loc[result_how_often_fold_3[i] == True]
        fscore_mut_fold_3 = mut_count_fscore_fold_3['DMS_score'].loc[mut_count_fscore_fold_3['mut_count'] == j]
        wie_viel_jeweils_fold_3 = len(fscore_mut_fold_3)
        how_many_for_variance_fold_3.append(wie_viel_jeweils_fold_3)

    how_many_for_variance_fold_3 = pd.Series(how_many_for_variance_fold_3, index=all_possible_mutations_fold_3)
    how_many_for_variance_df_fold_3 = how_many_for_variance_fold_3.to_frame()

    how_many_AND_variance_df_fold_3 = pd.concat([how_many_for_variance_df_fold_3, variance_per_mutant_df_fold_3],
                                                axis=1)
    how_many_AND_variance_df_fold_3.columns = ['Anzahl benutzter Werte', 'Varianz']
    how_many_AND_variance_df_fold_3 = how_many_AND_variance_df_fold_3.dropna()
#calculating the mean of the variances across all mutation counts for every possible mutation
frame_zum_mitteln_variance_fold_3 = pd.DataFrame(index=all_possible_mutations_fold_3)
variance_per_mutant_count_list_fold_3 = []

# ACHTUNG: es werden nur counts von 2 bis 7 einbezogen weil die mit mehr sowieso "kaputt" sind!!!
for j in range(2, 8):
    variance_per_mutant_list_fold_3 = []

    for i in all_possible_mutations_fold_3:
        mut_count_fscore_fold_3 = count_fscore_frame_fold_3.loc[result_how_often_fold_3[i] == True]
        fscore_mut_fold_3 = mut_count_fscore_fold_3['DMS_score'].loc[mut_count_fscore_fold_3['mut_count'] == j]
        varianz_mut_fold_3 = fscore_mut_fold_3.var()  #die varianz je mutation je anzahl
        variance_per_mutant_list_fold_3.append(varianz_mut_fold_3)  #liste der Varianzen ALLER Mutationen je anzahl

    variance_per_mutant_df = pd.DataFrame(variance_per_mutant_list_fold_3, index=all_possible_mutations_fold_3)
    variance_per_mutant_count_list_fold_3.append(variance_per_mutant_df)
variance_per_mutant_count_df_fold_3 = pd.concat(variance_per_mutant_count_list_fold_3, axis=1)
variance_per_mutant_count_df_fold_3.set_axis(range(2, 8), axis=1, inplace=True)

# dataframe mit allen varianzen (Zellen) pro alle mutationen (rows) pro alle counts (columns)
mean_variances_per_mutations_fold_3 = pd.DataFrame(variance_per_mutant_count_df_fold_3.mean(axis=1, skipna=True),
                                                   columns=['Mean'])

how_many_per_mutant_count_list_fold_3 = []

for j in range(2, 8):
    how_many_for_variance_fold_3 = []

    for i in all_possible_mutations_fold_3:
        mut_count_fscore_fold_3 = count_fscore_frame_fold_3.loc[result_how_often_fold_3[i] == True]
        fscore_mut_fold_3 = mut_count_fscore_fold_3['DMS_score'].loc[mut_count_fscore_fold_3['mut_count'] == j]
        wie_viel_jeweils_fold_3 = len(fscore_mut_fold_3)
        how_many_for_variance_fold_3.append(wie_viel_jeweils_fold_3)

    how_many_per_mutant_df_fold_3 = pd.DataFrame(how_many_for_variance_fold_3, index=all_possible_mutations_fold_3)
    how_many_per_mutant_count_list_fold_3.append(how_many_per_mutant_df_fold_3)
how_many_per_mutant_count_df_fold_3 = pd.concat(how_many_per_mutant_count_list_fold_3, axis=1)
how_many_per_mutant_count_df_fold_3.set_axis(range(2, 8), axis=1, inplace=True)

mean_how_many_per_mutations_fold_3 = pd.DataFrame(how_many_per_mutant_count_df_fold_3.mean(axis=1, skipna=True),
                                                  columns=['Mean'])
#differences between all mutant-fscores WITH and WITHOUT the one specific mutation, takes loong
nur_fscore_mut_count_fold_3 = working_dataframe_fold_3.loc[:, ["DMS_score", "mut_count"]]
differences_list_fold_3 = []

for i in all_possible_mutations_fold_3:
    index_when_mut_present_fold_3 = result_how_often_fold_3.loc[result_how_often_fold_3[i] == True].index

    only_rows_with_mut_fold_3 = nur_fscore_mut_count_fold_3[(nur_fscore_mut_count_fold_3['mut_count'] > 2) & (
        nur_fscore_mut_count_fold_3.index.isin(index_when_mut_present_fold_3))]

    # Calculate the mean of DMS_score for the filtered rows
    mean_dms_score_only_mut_fold_3 = only_rows_with_mut_fold_3['DMS_score'].mean()
    #-------------
    index_when_not_mut_present_fold_3 = result_how_often_fold_3.loc[result_how_often_fold_3[i] == False].index

    only_rows_withOUT_mut_fold_3 = working_dataframe_fold_3[(working_dataframe_fold_3['mut_count'] > 2) & (
        working_dataframe_fold_3.index.isin(index_when_not_mut_present_fold_3))]

    # Calculate the mean of DMS_score for the filtered rows
    mean_dms_score_every_but_mut_fold_3 = only_rows_withOUT_mut_fold_3['DMS_score'].mean()
    #----------------
    difference_means_fold_3 = mean_dms_score_only_mut_fold_3 - mean_dms_score_every_but_mut_fold_3
    differences_list_fold_3.append(difference_means_fold_3)

all_differences_means_fold_3 = pd.DataFrame({'Difference': differences_list_fold_3},
                                            index=all_possible_mutations_fold_3)

list_wie_oft_mut_fold_3 = []
for j in all_possible_mutations_fold_3:
    matching_indexes_fold_3 = result_how_often_fold_3.loc[result_how_often_fold_3[j] == True].index
    wie_oft_fold_3 = len(matching_indexes_fold_3)
    list_wie_oft_mut_fold_3.append(wie_oft_fold_3)
df_wie_oft_muts_insg_fold_3 = pd.DataFrame(list_wie_oft_mut_fold_3, index=all_possible_mutations_fold_3)
df_wie_oft_muts_insg_fold_3
list_ranking5_fold_3 = []
for i in all_possible_mutations_fold_3:
    score_ranking5_fold_3 = (all_differences_means_fold_3.loc[i].values[0] * df_wie_oft_muts_insg_fold_3.loc[i].values[
        0] * 51714) / np.sum(variance_per_mutant_count_df_fold_3.loc[i].values)
    list_ranking5_fold_3.append(score_ranking5_fold_3)
ranking5_unsorted_fold_3 = pd.DataFrame(list_ranking5_fold_3, index=all_possible_mutations_fold_3,
                                        columns=['ranking5_score_fold_3'])
ranking5_fold_3 = ranking5_unsorted_fold_3.sort_values(by='ranking5_score_fold_3', ascending=False)
print(ranking5_fold_3)



RangeIndex(start=0, stop=10342, step=1)
RangeIndex(start=0, stop=10342, step=1)


/var/folders/sk/q8h0c3zs1v9dt93y5m134swr0000gn/T/ipykernel_11103/467366181.py:107: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  variance_per_mutant_count_df_fold_3.set_axis(range(2, 8), axis=1, inplace=True)
/var/folders/sk/q8h0c3zs1v9dt93y5m134swr0000gn/T/ipykernel_11103/467366181.py:127: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  how_many_per_mutant_count_df_fold_3.set_axis(range(2, 8), axis=1, inplace=True)


       ranking5_score_fold_3
I171V           1.239958e+06
V163A           1.154517e+06
S175G           8.812701e+05
K113R           6.875590e+05
G10R            6.741375e+05
...                      ...
L201V                    NaN
G20A                     NaN
K131T                    NaN
D76A                     NaN
P58L                     NaN

[1469 rows x 1 columns]




--------------fold 4--------------

In [198]:
# Assuming your DataFrame is named 'fold_4_dms_col'
fold_4_dms = fold_4_dms.copy()  # Make a copy of the DataFrame

# Split the 'mutant' column into multiple columns
split_columns_fold_4 = fold_4_dms['mutant'].str.split(':', expand=True)

# Generate column names from 'm1' to 'mN'
num_columns_fold_4 = split_columns_fold_4.shape[1]
column_names_fold_4 = [f"m{i+1}" for i in range(num_columns_fold_4)]

# Assign the split columns with the generated column names to the DataFrame
fold_4_dms = pd.concat([fold_4_dms, split_columns_fold_4], axis=1)
fold_4_dms.columns = list(fold_4_dms.columns[:len(fold_4_dms.columns)-num_columns_fold_4]) + column_names_fold_4

fold_4_dms = fold_4_dms.reset_index()
fold_4_dms = fold_4_dms.drop('index', axis=1)  # Remove the old index column if needed

print(fold_4_dms.index)
# how many mutations does each sequence contain?
list_mut_count_in_progress_fold_4 = []
for i in range(len(fold_4_dms['mutant'])):
    list_mut_count_in_progress_fold_4.append(fold_4_dms['mutant'].iloc[i].count(':'))
list_mut_count_prae_fold_4 = np.array(list_mut_count_in_progress_fold_4)
list_mut_count_fold_4 = (list_mut_count_prae_fold_4 + 1)
df_mutation_counts_fold_4 = pd.DataFrame(list_mut_count_fold_4)
print(df_mutation_counts_fold_4.index)
working_dataframe_fold_4_try = pd.concat([fold_4_dms, df_mutation_counts_fold_4], axis=1)
#more convenient df
working_dataframe_fold_4 = pd.concat([fold_4_dms, df_mutation_counts_fold_4[0]], axis=1)
columns_to_drop = ['mutant', 'mutated_sequence', 'DMS_score_bin']
working_dataframe_fold_4.drop(columns_to_drop, axis=1)
working_dataframe_fold_4.rename(columns={0: 'mut_count'}, inplace=True)
count_fscore_frame_fold_4 = working_dataframe_fold_4[['DMS_score', 'mut_count']]
count_fscore_frame_fold_4.head()
#generating list that contains all possible mutations
import re

mutation_columns_fold_4 = [col for col in working_dataframe_fold_4.columns if re.match(r'^m\d+$', col)]
working_dataframe_only_ms_fold_4 = working_dataframe_fold_4.loc[:, mutation_columns_fold_4]
stacked_columns_fold_4 = working_dataframe_only_ms_fold_4.stack().tolist()
all_possible_mutations_fold_4 = list(set(stacked_columns_fold_4))
if 'None' in all_possible_mutations_fold_4:
    all_possible_mutations_fold_4.remove('None')

#dataframe which mutants contain the mutation
list_of_dfs_fold_4 = []

# For-Loop zum Erstellen und Hinzufügen eines neuen Columns in jedem Durchlauf
for i in all_possible_mutations_fold_4:
    new_column_name_fold_4 = f'{i}'
    new_column_values_fold_4 = [fold_4_dms["mutant"].str.contains(i, regex=False)]
    new_df_fold_4 = pd.DataFrame({new_column_name_fold_4: new_column_values_fold_4})
    new_df_exploded_fold_4 = new_df_fold_4.explode(new_column_name_fold_4)
    list_of_dfs_fold_4.append(new_df_exploded_fold_4)

# Verketten der DataFrames in der Liste
result_how_often_fold_4 = pd.concat(list_of_dfs_fold_4, axis=1)
result_how_often_fold_4 = result_how_often_fold_4.reset_index(drop=True)

#Varianz berechnen lassen, mit wie vielen Werten wurde gerechnet

for j in range(2, 16):
    variance_per_mutant_list_fold_4 = []

    for i in all_possible_mutations_fold_4:
        mut_count_fscore_fold_4 = count_fscore_frame_fold_4.loc[result_how_often_fold_4[i] == True]
        fscore_mut_fold_4 = mut_count_fscore_fold_4['DMS_score'].loc[mut_count_fscore_fold_4['mut_count'] == j]
        varianz_mut_fold_4 = fscore_mut_fold_4.var()
        variance_per_mutant_list_fold_4.append(varianz_mut_fold_4)

    variance_per_mutant_series_fold_4 = pd.Series(variance_per_mutant_list_fold_4, index=all_possible_mutations_fold_4)
    variance_per_mutant_df_fold_4 = variance_per_mutant_series_fold_4.to_frame()

    #rausfinden wie viele Daten wir jeweils haben zum berechnen
    how_many_for_variance_fold_4 = []

    for i in all_possible_mutations_fold_4:
        mut_count_fscore_fold_4 = count_fscore_frame_fold_4.loc[result_how_often_fold_4[i] == True]
        fscore_mut_fold_4 = mut_count_fscore_fold_4['DMS_score'].loc[mut_count_fscore_fold_4['mut_count'] == j]
        wie_viel_jeweils_fold_4 = len(fscore_mut_fold_4)
        how_many_for_variance_fold_4.append(wie_viel_jeweils_fold_4)

    how_many_for_variance_fold_4 = pd.Series(how_many_for_variance_fold_4, index=all_possible_mutations_fold_4)
    how_many_for_variance_df_fold_4 = how_many_for_variance_fold_4.to_frame()

    how_many_AND_variance_df_fold_4 = pd.concat([how_many_for_variance_df_fold_4, variance_per_mutant_df_fold_4],
                                                axis=1)
    how_many_AND_variance_df_fold_4.columns = ['Anzahl benutzter Werte', 'Varianz']
    how_many_AND_variance_df_fold_4 = how_many_AND_variance_df_fold_4.dropna()
#calculating the mean of the variances across all mutation counts for every possible mutation
frame_zum_mitteln_variance_fold_4 = pd.DataFrame(index=all_possible_mutations_fold_4)
variance_per_mutant_count_list_fold_4 = []

# ACHTUNG: es werden nur counts von 2 bis 7 einbezogen weil die mit mehr sowieso "kaputt" sind!!!
for j in range(2, 8):
    variance_per_mutant_list_fold_4 = []

    for i in all_possible_mutations_fold_4:
        mut_count_fscore_fold_4 = count_fscore_frame_fold_4.loc[result_how_often_fold_4[i] == True]
        fscore_mut_fold_4 = mut_count_fscore_fold_4['DMS_score'].loc[mut_count_fscore_fold_4['mut_count'] == j]
        varianz_mut_fold_4 = fscore_mut_fold_4.var()  #die varianz je mutation je anzahl
        variance_per_mutant_list_fold_4.append(varianz_mut_fold_4)  #liste der Varianzen ALLER Mutationen je anzahl

    variance_per_mutant_df = pd.DataFrame(variance_per_mutant_list_fold_4, index=all_possible_mutations_fold_4)
    variance_per_mutant_count_list_fold_4.append(variance_per_mutant_df)
variance_per_mutant_count_df_fold_4 = pd.concat(variance_per_mutant_count_list_fold_4, axis=1)
variance_per_mutant_count_df_fold_4.set_axis(range(2, 8), axis=1, inplace=True)

# dataframe mit allen varianzen (Zellen) pro alle mutationen (rows) pro alle counts (columns)
mean_variances_per_mutations_fold_4 = pd.DataFrame(variance_per_mutant_count_df_fold_4.mean(axis=1, skipna=True),
                                                   columns=['Mean'])

how_many_per_mutant_count_list_fold_4 = []

for j in range(2, 8):
    how_many_for_variance_fold_4 = []

    for i in all_possible_mutations_fold_4:
        mut_count_fscore_fold_4 = count_fscore_frame_fold_4.loc[result_how_often_fold_4[i] == True]
        fscore_mut_fold_4 = mut_count_fscore_fold_4['DMS_score'].loc[mut_count_fscore_fold_4['mut_count'] == j]
        wie_viel_jeweils_fold_4 = len(fscore_mut_fold_4)
        how_many_for_variance_fold_4.append(wie_viel_jeweils_fold_4)

    how_many_per_mutant_df_fold_4 = pd.DataFrame(how_many_for_variance_fold_4, index=all_possible_mutations_fold_4)
    how_many_per_mutant_count_list_fold_4.append(how_many_per_mutant_df_fold_4)
how_many_per_mutant_count_df_fold_4 = pd.concat(how_many_per_mutant_count_list_fold_4, axis=1)
how_many_per_mutant_count_df_fold_4.set_axis(range(2, 8), axis=1, inplace=True)

mean_how_many_per_mutations_fold_4 = pd.DataFrame(how_many_per_mutant_count_df_fold_4.mean(axis=1, skipna=True),
                                                  columns=['Mean'])
#differences between all mutant-fscores WITH and WITHOUT the one specific mutation, takes loong
nur_fscore_mut_count_fold_4 = working_dataframe_fold_4.loc[:, ["DMS_score", "mut_count"]]
differences_list_fold_4 = []

for i in all_possible_mutations_fold_4:
    index_when_mut_present_fold_4 = result_how_often_fold_4.loc[result_how_often_fold_4[i] == True].index

    only_rows_with_mut_fold_4 = nur_fscore_mut_count_fold_4[(nur_fscore_mut_count_fold_4['mut_count'] > 2) & (
        nur_fscore_mut_count_fold_4.index.isin(index_when_mut_present_fold_4))]

    # Calculate the mean of DMS_score for the filtered rows
    mean_dms_score_only_mut_fold_4 = only_rows_with_mut_fold_4['DMS_score'].mean()
    #-------------
    index_when_not_mut_present_fold_4 = result_how_often_fold_4.loc[result_how_often_fold_4[i] == False].index

    only_rows_withOUT_mut_fold_4 = working_dataframe_fold_4[(working_dataframe_fold_4['mut_count'] > 2) & (
        working_dataframe_fold_4.index.isin(index_when_not_mut_present_fold_4))]

    # Calculate the mean of DMS_score for the filtered rows
    mean_dms_score_every_but_mut_fold_4 = only_rows_withOUT_mut_fold_4['DMS_score'].mean()
    #----------------
    difference_means_fold_4 = mean_dms_score_only_mut_fold_4 - mean_dms_score_every_but_mut_fold_4
    differences_list_fold_4.append(difference_means_fold_4)

all_differences_means_fold_4 = pd.DataFrame({'Difference': differences_list_fold_4},
                                            index=all_possible_mutations_fold_4)

list_wie_oft_mut_fold_4 = []
for j in all_possible_mutations_fold_4:
    matching_indexes_fold_4 = result_how_often_fold_4.loc[result_how_often_fold_4[j] == True].index
    wie_oft_fold_4 = len(matching_indexes_fold_4)
    list_wie_oft_mut_fold_4.append(wie_oft_fold_4)
df_wie_oft_muts_insg_fold_4 = pd.DataFrame(list_wie_oft_mut_fold_4, index=all_possible_mutations_fold_4)
df_wie_oft_muts_insg_fold_4
list_ranking5_fold_4 = []
for i in all_possible_mutations_fold_4:
    score_ranking5_fold_4 = (all_differences_means_fold_4.loc[i].values[0] * df_wie_oft_muts_insg_fold_4.loc[i].values[
        0] * 51714) / np.sum(variance_per_mutant_count_df_fold_4.loc[i].values)
    list_ranking5_fold_4.append(score_ranking5_fold_4)
ranking5_unsorted_fold_4 = pd.DataFrame(list_ranking5_fold_4, index=all_possible_mutations_fold_4,
                                        columns=['ranking5_score_fold_4'])
ranking5_fold_4 = ranking5_unsorted_fold_4.sort_values(by='ranking5_score_fold_4', ascending=False)
print(ranking5_fold_4)


RangeIndex(start=0, stop=10342, step=1)
RangeIndex(start=0, stop=10342, step=1)


/var/folders/sk/q8h0c3zs1v9dt93y5m134swr0000gn/T/ipykernel_11103/3550614773.py:107: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  variance_per_mutant_count_df_fold_4.set_axis(range(2, 8), axis=1, inplace=True)
/var/folders/sk/q8h0c3zs1v9dt93y5m134swr0000gn/T/ipykernel_11103/3550614773.py:127: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  how_many_per_mutant_count_df_fold_4.set_axis(range(2, 8), axis=1, inplace=True)


       ranking5_score_fold_4
V163A          910749.941437
D129G          882110.613644
I171T          704941.387390
K214E          663477.387878
I171V          648781.511224
...                      ...
R215S                    NaN
G91R                     NaN
D76A                     NaN
P58L                     NaN
N23C                     NaN

[1461 rows x 1 columns]




--------------fold 5-------------------

In [199]:
# Assuming your DataFrame is named 'fold_5_dms_col'
fold_5_dms = fold_5_dms.copy()  # Make a copy of the DataFrame

# Split the 'mutant' column into multiple columns
split_columns_fold_5 = fold_5_dms['mutant'].str.split(':', expand=True)

# Generate column names from 'm1' to 'mN'
num_columns_fold_5 = split_columns_fold_5.shape[1]
column_names_fold_5 = [f"m{i+1}" for i in range(num_columns_fold_5)]

# Assign the split columns with the generated column names to the DataFrame
fold_5_dms = pd.concat([fold_5_dms, split_columns_fold_5], axis=1)
fold_5_dms.columns = list(fold_5_dms.columns[:len(fold_5_dms.columns)-num_columns_fold_5]) + column_names_fold_5

fold_5_dms = fold_5_dms.reset_index()
fold_5_dms = fold_5_dms.drop('index', axis=1)  # Remove the old index column if needed

print(fold_5_dms.index)
# how many mutations does each sequence contain?
list_mut_count_in_progress_fold_5 = []
for i in range(len(fold_5_dms['mutant'])):
    list_mut_count_in_progress_fold_5.append(fold_5_dms['mutant'].iloc[i].count(':'))
list_mut_count_prae_fold_5 = np.array(list_mut_count_in_progress_fold_5)
list_mut_count_fold_5 = (list_mut_count_prae_fold_5 + 1)
df_mutation_counts_fold_5 = pd.DataFrame(list_mut_count_fold_5)
print(df_mutation_counts_fold_5.index)
working_dataframe_fold_5_try = pd.concat([fold_5_dms, df_mutation_counts_fold_5], axis=1)
#more convenient df
working_dataframe_fold_5 = pd.concat([fold_5_dms, df_mutation_counts_fold_5[0]], axis=1)
columns_to_drop = ['mutant', 'mutated_sequence', 'DMS_score_bin']
working_dataframe_fold_5.drop(columns_to_drop, axis=1)
working_dataframe_fold_5.rename(columns={0: 'mut_count'}, inplace=True)
count_fscore_frame_fold_5 = working_dataframe_fold_5[['DMS_score', 'mut_count']]
count_fscore_frame_fold_5.head()
#generating list that contains all possible mutations
import re

mutation_columns_fold_5 = [col for col in working_dataframe_fold_5.columns if re.match(r'^m\d+$', col)]
working_dataframe_only_ms_fold_5 = working_dataframe_fold_5.loc[:, mutation_columns_fold_5]
stacked_columns_fold_5 = working_dataframe_only_ms_fold_5.stack().tolist()
all_possible_mutations_fold_5 = list(set(stacked_columns_fold_5))
if 'None' in all_possible_mutations_fold_5:
    all_possible_mutations_fold_5.remove('None')

#dataframe which mutants contain the mutation
list_of_dfs_fold_5 = []

# For-Loop zum Erstellen und Hinzufügen eines neuen Columns in jedem Durchlauf
for i in all_possible_mutations_fold_5:
    new_column_name_fold_5 = f'{i}'
    new_column_values_fold_5 = [fold_5_dms["mutant"].str.contains(i, regex=False)]
    new_df_fold_5 = pd.DataFrame({new_column_name_fold_5: new_column_values_fold_5})
    new_df_exploded_fold_5 = new_df_fold_5.explode(new_column_name_fold_5)
    list_of_dfs_fold_5.append(new_df_exploded_fold_5)

# Verketten der DataFrames in der Liste
result_how_often_fold_5 = pd.concat(list_of_dfs_fold_5, axis=1)
result_how_often_fold_5 = result_how_often_fold_5.reset_index(drop=True)

#Varianz berechnen lassen, mit wie vielen Werten wurde gerechnet

for j in range(2, 16):
    variance_per_mutant_list_fold_5 = []

    for i in all_possible_mutations_fold_5:
        mut_count_fscore_fold_5 = count_fscore_frame_fold_5.loc[result_how_often_fold_5[i] == True]
        fscore_mut_fold_5 = mut_count_fscore_fold_5['DMS_score'].loc[mut_count_fscore_fold_5['mut_count'] == j]
        varianz_mut_fold_5 = fscore_mut_fold_5.var()
        variance_per_mutant_list_fold_5.append(varianz_mut_fold_5)

    variance_per_mutant_series_fold_5 = pd.Series(variance_per_mutant_list_fold_5, index=all_possible_mutations_fold_5)
    variance_per_mutant_df_fold_5 = variance_per_mutant_series_fold_5.to_frame()

    #rausfinden wie viele Daten wir jeweils haben zum berechnen
    how_many_for_variance_fold_5 = []

    for i in all_possible_mutations_fold_5:
        mut_count_fscore_fold_5 = count_fscore_frame_fold_5.loc[result_how_often_fold_5[i] == True]
        fscore_mut_fold_5 = mut_count_fscore_fold_5['DMS_score'].loc[mut_count_fscore_fold_5['mut_count'] == j]
        wie_viel_jeweils_fold_5 = len(fscore_mut_fold_5)
        how_many_for_variance_fold_5.append(wie_viel_jeweils_fold_5)

    how_many_for_variance_fold_5 = pd.Series(how_many_for_variance_fold_5, index=all_possible_mutations_fold_5)
    how_many_for_variance_df_fold_5 = how_many_for_variance_fold_5.to_frame()

    how_many_AND_variance_df_fold_5 = pd.concat([how_many_for_variance_df_fold_5, variance_per_mutant_df_fold_5],
                                                axis=1)
    how_many_AND_variance_df_fold_5.columns = ['Anzahl benutzter Werte', 'Varianz']
    how_many_AND_variance_df_fold_5 = how_many_AND_variance_df_fold_5.dropna()
#calculating the mean of the variances across all mutation counts for every possible mutation
frame_zum_mitteln_variance_fold_5 = pd.DataFrame(index=all_possible_mutations_fold_5)
variance_per_mutant_count_list_fold_5 = []

# ACHTUNG: es werden nur counts von 2 bis 7 einbezogen weil die mit mehr sowieso "kaputt" sind!!!
for j in range(2, 8):
    variance_per_mutant_list_fold_5 = []

    for i in all_possible_mutations_fold_5:
        mut_count_fscore_fold_5 = count_fscore_frame_fold_5.loc[result_how_often_fold_5[i] == True]
        fscore_mut_fold_5 = mut_count_fscore_fold_5['DMS_score'].loc[mut_count_fscore_fold_5['mut_count'] == j]
        varianz_mut_fold_5 = fscore_mut_fold_5.var()  #die varianz je mutation je anzahl
        variance_per_mutant_list_fold_5.append(varianz_mut_fold_5)  #liste der Varianzen ALLER Mutationen je anzahl

    variance_per_mutant_df = pd.DataFrame(variance_per_mutant_list_fold_5, index=all_possible_mutations_fold_5)
    variance_per_mutant_count_list_fold_5.append(variance_per_mutant_df)
variance_per_mutant_count_df_fold_5 = pd.concat(variance_per_mutant_count_list_fold_5, axis=1)
variance_per_mutant_count_df_fold_5.set_axis(range(2, 8), axis=1, inplace=True)

# dataframe mit allen varianzen (Zellen) pro alle mutationen (rows) pro alle counts (columns)
mean_variances_per_mutations_fold_5 = pd.DataFrame(variance_per_mutant_count_df_fold_5.mean(axis=1, skipna=True),
                                                   columns=['Mean'])

how_many_per_mutant_count_list_fold_5 = []

for j in range(2, 8):
    how_many_for_variance_fold_5 = []

    for i in all_possible_mutations_fold_5:
        mut_count_fscore_fold_5 = count_fscore_frame_fold_5.loc[result_how_often_fold_5[i] == True]
        fscore_mut_fold_5 = mut_count_fscore_fold_5['DMS_score'].loc[mut_count_fscore_fold_5['mut_count'] == j]
        wie_viel_jeweils_fold_5 = len(fscore_mut_fold_5)
        how_many_for_variance_fold_5.append(wie_viel_jeweils_fold_5)

    how_many_per_mutant_df_fold_5 = pd.DataFrame(how_many_for_variance_fold_5, index=all_possible_mutations_fold_5)
    how_many_per_mutant_count_list_fold_5.append(how_many_per_mutant_df_fold_5)
how_many_per_mutant_count_df_fold_5 = pd.concat(how_many_per_mutant_count_list_fold_5, axis=1)
how_many_per_mutant_count_df_fold_5.set_axis(range(2, 8), axis=1, inplace=True)

mean_how_many_per_mutations_fold_5 = pd.DataFrame(how_many_per_mutant_count_df_fold_5.mean(axis=1, skipna=True),
                                                  columns=['Mean'])
#differences between all mutant-fscores WITH and WITHOUT the one specific mutation, takes loong
nur_fscore_mut_count_fold_5 = working_dataframe_fold_5.loc[:, ["DMS_score", "mut_count"]]
differences_list_fold_5 = []

for i in all_possible_mutations_fold_5:
    index_when_mut_present_fold_5 = result_how_often_fold_5.loc[result_how_often_fold_5[i] == True].index

    only_rows_with_mut_fold_5 = nur_fscore_mut_count_fold_5[(nur_fscore_mut_count_fold_5['mut_count'] > 2) & (
        nur_fscore_mut_count_fold_5.index.isin(index_when_mut_present_fold_5))]

    # Calculate the mean of DMS_score for the filtered rows
    mean_dms_score_only_mut_fold_5 = only_rows_with_mut_fold_5['DMS_score'].mean()
    #-------------
    index_when_not_mut_present_fold_5 = result_how_often_fold_5.loc[result_how_often_fold_5[i] == False].index

    only_rows_withOUT_mut_fold_5 = working_dataframe_fold_5[(working_dataframe_fold_5['mut_count'] > 2) & (
        working_dataframe_fold_5.index.isin(index_when_not_mut_present_fold_5))]

    # Calculate the mean of DMS_score for the filtered rows
    mean_dms_score_every_but_mut_fold_5 = only_rows_withOUT_mut_fold_5['DMS_score'].mean()
    #----------------
    difference_means_fold_5 = mean_dms_score_only_mut_fold_5 - mean_dms_score_every_but_mut_fold_5
    differences_list_fold_5.append(difference_means_fold_5)

all_differences_means_fold_5 = pd.DataFrame({'Difference': differences_list_fold_5},
                                            index=all_possible_mutations_fold_5)

list_wie_oft_mut_fold_5 = []
for j in all_possible_mutations_fold_5:
    matching_indexes_fold_5 = result_how_often_fold_5.loc[result_how_often_fold_5[j] == True].index
    wie_oft_fold_5 = len(matching_indexes_fold_5)
    list_wie_oft_mut_fold_5.append(wie_oft_fold_5)
df_wie_oft_muts_insg_fold_5 = pd.DataFrame(list_wie_oft_mut_fold_5, index=all_possible_mutations_fold_5)
df_wie_oft_muts_insg_fold_5
list_ranking5_fold_5 = []
for i in all_possible_mutations_fold_5:
    score_ranking5_fold_5 = (all_differences_means_fold_5.loc[i].values[0] * df_wie_oft_muts_insg_fold_5.loc[i].values[
        0] * 51714) / np.sum(variance_per_mutant_count_df_fold_5.loc[i].values)
    list_ranking5_fold_5.append(score_ranking5_fold_5)
ranking5_unsorted_fold_5 = pd.DataFrame(list_ranking5_fold_5, index=all_possible_mutations_fold_5,
                                        columns=['ranking5_score_fold_5'])
ranking5_fold_5 = ranking5_unsorted_fold_5.sort_values(by='ranking5_score_fold_5', ascending=False)
print(ranking5_fold_5)



RangeIndex(start=0, stop=10346, step=1)
RangeIndex(start=0, stop=10346, step=1)


/var/folders/sk/q8h0c3zs1v9dt93y5m134swr0000gn/T/ipykernel_11103/357612555.py:107: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  variance_per_mutant_count_df_fold_5.set_axis(range(2, 8), axis=1, inplace=True)
/var/folders/sk/q8h0c3zs1v9dt93y5m134swr0000gn/T/ipykernel_11103/357612555.py:127: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  how_many_per_mutant_count_df_fold_5.set_axis(range(2, 8), axis=1, inplace=True)


       ranking5_score_fold_5
K156E           1.016729e+06
V163A           9.711327e+05
I167V           7.691295e+05
I123V           7.228442e+05
N144D           6.780198e+05
...                      ...
R215S                    NaN
F165I                    NaN
G91R                     NaN
D76A                     NaN
P58L                     NaN

[1485 rows x 1 columns]





-----spearman´s korrelation----------

In [214]:
from scipy.stats import spearmanr
from itertools import combinations

desired_length = 10

rankings = [ranking5_fold_1, ranking5_fold_2, ranking5_fold_3, ranking5_fold_4, ranking5_fold_5]

def calculate_spearman_correlation(rank1, rank2):
    # Convert DataFrame indexes to lists
    rank1 = rank1.index.tolist()
    rank2 = rank2.index.tolist()

    # Calculate Spearman's correlation coefficient
    correlation, p_value = spearmanr(rank1, rank2)

    return correlation, p_value

# Store the results in a list of dictionaries
results = []
fold_numbers = list(range(1, len(rankings) + 1))

# Calculate Spearman's correlation for every combination of rankings
for combo in combinations(enumerate(rankings, 1), 2):
    fold1_num, fold1_rank = combo[0]
    fold2_num, fold2_rank = combo[1]

    fold1_truncated = fold1_rank.head(desired_length)
    fold2_truncated = fold2_rank.head(desired_length)

    correlation, p_value = calculate_spearman_correlation(fold1_truncated, fold2_truncated)

    result = {
        'Fold1': fold1_num,
        'Fold2': fold2_num,
        'Correlation': correlation,
        'P-value': p_value
    }
    results.append(result)

# Print the results
for result in results:
    print("Fold", result['Fold1'], "vs Fold", result['Fold2'])
    print("Spearman's correlation coefficient:", result['Correlation'])
    print("p-value:", result['P-value'])
    print()


Fold 1 vs Fold 2
Spearman's correlation coefficient: 0.6484848484848483
p-value: 0.04254012768448945

Fold 1 vs Fold 3
Spearman's correlation coefficient: -0.18787878787878787
p-value: 0.603217610029209

Fold 1 vs Fold 4
Spearman's correlation coefficient: 0.43030303030303024
p-value: 0.21449233253280775

Fold 1 vs Fold 5
Spearman's correlation coefficient: -0.34545454545454546
p-value: 0.32822651147136733

Fold 2 vs Fold 3
Spearman's correlation coefficient: 0.01818181818181818
p-value: 0.9602404181286243

Fold 2 vs Fold 4
Spearman's correlation coefficient: 0.309090909090909
p-value: 0.3848412302612835

Fold 2 vs Fold 5
Spearman's correlation coefficient: -0.2121212121212121
p-value: 0.5563057751029299

Fold 3 vs Fold 4
Spearman's correlation coefficient: -0.5272727272727272
p-value: 0.11730806555020223

Fold 3 vs Fold 5
Spearman's correlation coefficient: -0.18787878787878787
p-value: 0.603217610029209

Fold 4 vs Fold 5
Spearman's correlation coefficient: -0.05454545454545454
p-valu

/Users/liza/anaconda3/envs/DMSproject/lib/python3.10/site-packages/scipy/stats/_stats_py.py:112: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "



--------Kendall´s tau-b correlation

In [236]:
from scipy.stats import kendalltau
from itertools import combinations

desired_length = 10
rankings = [ranking5_fold_1, ranking5_fold_2, ranking5_fold_3, ranking5_fold_4, ranking5_fold_5]

def calculate_kendall_correlation(rank1, rank2):
    # Convert DataFrame indexes to lists
    rank1 = rank1.index.tolist()
    rank2 = rank2.index.tolist()

    # Calculate Kendall's correlation coefficient
    correlation, p_value = kendalltau(rank1, rank2)

    return correlation, p_value

# Store the results in a list of dictionaries
results = []
fold_numbers = list(range(1, len(rankings) + 1))

# Calculate Kendall's correlation for every combination of rankings
for combo in combinations(enumerate(rankings, 1), 2):
    fold1_num, fold1_rank = combo[0]
    fold2_num, fold2_rank = combo[1]

    fold1_truncated = fold1_rank.head(desired_length)
    fold2_truncated = fold2_rank.head(desired_length)

    correlation, p_value = calculate_kendall_correlation(fold1_truncated, fold2_truncated)

    result = {
        'Fold1': fold1_num,
        'Fold2': fold2_num,
        'Correlation': correlation,
        'P-value': p_value
    }
    results.append(result)

# Print the results
for result in results:
    print("Fold", result['Fold1'], "vs Fold", result['Fold2'])
    print("Kendall's correlation coefficient:", result['Correlation'])
    print("p-value:", result['P-value'])
    print()


Fold 1 vs Fold 2
Kendall's correlation coefficient: 0.5111111111111111
p-value: 0.04662257495590829

Fold 1 vs Fold 3
Kendall's correlation coefficient: -0.19999999999999998
p-value: 0.4843127204585538

Fold 1 vs Fold 4
Kendall's correlation coefficient: 0.37777777777777777
p-value: 0.1557418430335097

Fold 1 vs Fold 5
Kendall's correlation coefficient: -0.28888888888888886
p-value: 0.2912483465608466

Fold 2 vs Fold 3
Kendall's correlation coefficient: 0.022222222222222223
p-value: 1.0

Fold 2 vs Fold 4
Kendall's correlation coefficient: 0.24444444444444444
p-value: 0.38071979717813054

Fold 2 vs Fold 5
Kendall's correlation coefficient: -0.15555555555555553
p-value: 0.6006536596119929

Fold 3 vs Fold 4
Kendall's correlation coefficient: -0.37777777777777777
p-value: 0.1557418430335097

Fold 3 vs Fold 5
Kendall's correlation coefficient: -0.06666666666666667
p-value: 0.8618005952380953

Fold 4 vs Fold 5
Kendall's correlation coefficient: -0.022222222222222223
p-value: 1.0



/Users/liza/anaconda3/envs/DMSproject/lib/python3.10/site-packages/scipy/stats/_stats_py.py:112: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "


-------------spearman´s footrule distance -----------

In [235]:
from scipy.stats import spearmanr
from itertools import combinations

desired_length = 10
rankings = [ranking5_fold_1, ranking5_fold_2, ranking5_fold_3, ranking5_fold_4, ranking5_fold_5]

def calculate_spearman_distance(rank1, rank2):
    # Convert DataFrame indexes to lists
    rank1 = rank1.index.tolist()
    rank2 = rank2.index.tolist()

    distance = 0

    # Calculate Spearman's footrule distance for existing values
    for value in rank1:
        if value in rank2:
            distance += abs(rank1.index(value) - rank2.index(value))

    return distance

# Store the results in a list of dictionaries
results = []
fold_numbers = list(range(1, len(rankings) + 1))

# Calculate Spearman's footrule distance for every combination of rankings
for combo in combinations(enumerate(rankings, 1), 2):
    fold1_num, fold1_rank = combo[0]
    fold2_num, fold2_rank = combo[1]

    fold1_truncated = fold1_rank.head(desired_length)
    fold2_truncated = fold2_rank.head(desired_length)

    distance = calculate_spearman_distance(fold1_truncated, fold2_truncated)

    result = {
        'Fold1': fold1_num,
        'Fold2': fold2_num,
        'Distance': distance
    }
    results.append(result)

# Print the results
for result in results:
    print("Fold", result['Fold1'], "vs Fold", result['Fold2'])
    print("Spearman's footrule distance:", result['Distance'])
    print()


Fold 1 vs Fold 2
Spearman's footrule distance: 6

Fold 1 vs Fold 3
Spearman's footrule distance: 9

Fold 1 vs Fold 4
Spearman's footrule distance: 4

Fold 1 vs Fold 5
Spearman's footrule distance: 6

Fold 2 vs Fold 3
Spearman's footrule distance: 4

Fold 2 vs Fold 4
Spearman's footrule distance: 6

Fold 2 vs Fold 5
Spearman's footrule distance: 13

Fold 3 vs Fold 4
Spearman's footrule distance: 15

Fold 3 vs Fold 5
Spearman's footrule distance: 16

Fold 4 vs Fold 5
Spearman's footrule distance: 6



-----------goodman´s kruskal´s gamma -------------------

In [234]:
from scipy.stats import kendalltau
from itertools import combinations

desired_length = 10
rankings = [ranking5_fold_1, ranking5_fold_2, ranking5_fold_3, ranking5_fold_4, ranking5_fold_5]

def calculate_gamma(rank1, rank2):
    # Convert DataFrame indexes to lists
    rank1 = rank1.index.tolist()
    rank2 = rank2.index.tolist()

    concordant_pairs = 0
    discordant_pairs = 0

    # Calculate the number of concordant and discordant pairs
    for i in range(len(rank1)):
        for j in range(i+1, len(rank2)):
            if (rank1[i] < rank1[j] and rank2[i] < rank2[j]) or (rank1[i] > rank1[j] and rank2[i] > rank2[j]):
                concordant_pairs += 1
            elif (rank1[i] < rank1[j] and rank2[i] > rank2[j]) or (rank1[i] > rank1[j] and rank2[i] < rank2[j]):
                discordant_pairs += 1

    # Calculate Goodman's Kruskal's gamma
    gamma = (concordant_pairs - discordant_pairs) / (concordant_pairs + discordant_pairs)

    return gamma

# Store the results in a list of dictionaries
results = []
fold_numbers = list(range(1, len(rankings) + 1))

# Calculate Goodman's Kruskal's gamma for every combination of rankings
for combo in combinations(enumerate(rankings, 1), 2):
    fold1_num, fold1_rank = combo[0]
    fold2_num, fold2_rank = combo[1]

    fold1_truncated = fold1_rank.head(desired_length)
    fold2_truncated = fold2_rank.head(desired_length)

    gamma = calculate_gamma(fold1_truncated, fold2_truncated)

    result = {
        'Fold1': fold1_num,
        'Fold2': fold2_num,
        'Gamma': gamma
    }
    results.append(result)

# Print the results
for result in results:
    print("Fold", result['Fold1'], "vs Fold", result['Fold2'])
    print("Goodman's Kruskal's gamma:", result['Gamma'])
    print()


Fold 1 vs Fold 2
Goodman's Kruskal's gamma: 0.5111111111111111

Fold 1 vs Fold 3
Goodman's Kruskal's gamma: -0.2

Fold 1 vs Fold 4
Goodman's Kruskal's gamma: 0.37777777777777777

Fold 1 vs Fold 5
Goodman's Kruskal's gamma: -0.28888888888888886

Fold 2 vs Fold 3
Goodman's Kruskal's gamma: 0.022222222222222223

Fold 2 vs Fold 4
Goodman's Kruskal's gamma: 0.24444444444444444

Fold 2 vs Fold 5
Goodman's Kruskal's gamma: -0.15555555555555556

Fold 3 vs Fold 4
Goodman's Kruskal's gamma: -0.37777777777777777

Fold 3 vs Fold 5
Goodman's Kruskal's gamma: -0.06666666666666667

Fold 4 vs Fold 5
Goodman's Kruskal's gamma: -0.022222222222222223

